### The code for collecting keywords of national urban heritages

PS: Package 'pkuseg' cannot be used under python 3.8 environment. 

In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
import re

p = pd.read_csv("gcj02towgs84ed listing.csv", encoding='utf-8') #read the urban heritages list
p_g = gpd.GeoDataFrame(p,geometry = gpd.points_from_xy(p.new_x,p.new_y),crs = 'EPSG:4326')
p_g = p_g.to_crs(4490)

airbnb = pd.read_csv("cleaned data airbnb.csv", encoding='utf-8') #read the cleaned Airbnb listing data
airbnb_g = gpd.GeoDataFrame(airbnb,geometry = gpd.points_from_xy(airbnb.longitude,airbnb.latitude),crs = 'EPSG:4326')
airbnb_g = airbnb_g.to_crs(4490)

def airbnb_sub(n):
    #Input the number of national urban heritages
    #Output a subset of 10 listings located within a 1.5km radius of the heritages
    x = [p_g.loc[n,'new_x']]
    y = [p_g.loc[n,'new_y']]
    s = gpd.GeoSeries.from_xy(x, y, crs="EPSG:4490")
    s = s.to_crs(4548) #for changing the unit to meter
    s_b = s.buffer(1500) 
    s = gpd.GeoDataFrame(s)
    s = s.rename(columns={0:'geometry'}).set_geometry('geometry')
    s_b = gpd.GeoDataFrame(s_b)
    s_b = s_b.rename(columns={0:'geometry'}).set_geometry('geometry')
    airbnb_g2 = airbnb_g.to_crs(4548)
    out_a = gpd.sjoin(airbnb_g2,s_b,how = "inner")
    plt.figure(1, figsize=(16,9))
    ax = s.plot(color = 'r', zorder=2,markersize = 5)
    ax1 = out_a.plot(color = 'black', zorder=2,markersize = 3, alpha = .8, ax=ax)
    s_b.plot(ax = ax1, zorder = 1, markersize=3, color='b', alpha = .5)
    plt.xticks([])
    plt.yticks([])
    plt.legend(['Historical Building','Listings(within 1.5km)'],fontsize=8)
    plt.grid(True, alpha=0.5)
    plt.show()
    
    #only contain the first 10 listings
    if len(out_a)>=10:
        out_a = out_a[:10]
        out_a = pd.DataFrame(out_a.drop(columns='geometry'))
    else:
        out_a = pd.DataFrame(out_a.drop(columns='geometry'))
    out_a = out_a.reset_index(drop=True)
        
    return out_a

def cut_sent(para):
    para = re.sub('<b>', ' ', para)
    para = re.sub('</b>', ' ', para)
    para = re.sub('<br />', ' ', para)
    para = re.sub('The space',' ', para)
    para = re.sub('Guest access', ' ', para)
    para = re.sub('Other things to note', ' ', para)
    para = re.sub('([。！？\?])([^”’])', r"\1\n\2", para)  # single character sentence break
    para = re.sub('(\.{6})([^”’])', r"\1\n\2", para)  # English ellipsis
    para = re.sub('(\…{2})([^”’])', r"\1\n\2", para)  # Chinese ellipsis
    para = re.sub('([。！？\?][”’])([^，。！？\?])', r'\1\n\2', para)
    para = para.rstrip()  
    return para.split("\n")


# pkuseg - 'tourism' model + User Defined Dictionary
import pkuseg
from datetime import datetime

def pk_tourism2(sent):
    
    seg  = pkuseg.pkuseg(model_name='tourism',user_dict = 'listing.txt' ,postag=True) #load the model
    out_p = []
    for w in sent:
        out_p.append(seg.cut(w))
    return out_p

if __name__ == '__main__':
    num = 54 # here to change the number of national urban heritages -- input 0-54
    print(p_g.loc[num,'listing_en'])
    airbnb_s = airbnb_sub(num) 

    sentence = []
    for n in range(len(airbnb_s)):
        sentence.append(cut_sent(airbnb_s.loc[n,'description']))

    o = []
    for sent in sentence:
        print(datetime.now())
        p = pk_tourism2(sent)
        o.append(p)
    
    s = []
    for oo in o:
        ss = []
        for ooo in oo:
            for oooo in ooo:
                if oooo[1] in ('n','s','f','j','nr','ns','nt','nz'):
                    ss.append(oooo)
        s.append(ss)
        
    #save in local document
    for ss in s:
        s_w = str(ss)+"\n"+"\n"
        with open("word segmentation result.txt","a", encoding='utf-8') as f:
            f.write(s_w)